In [2]:
from openai import OpenAI
from langsmith import wrappers, traceable

openai = wrappers.wrap_openai(OpenAI())

In [3]:
import json

tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_movies",
            "description": "Retrieve a list of relevant movies and their metadata from a movie database.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query used to retrieve movies from the movie database, for example 'Christopher Nolan films'",
                    },
                },
                "required": ["query"],
            },
        },
    },
]

system_prompt = """
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
Note that if the question does not require additional search and can be answered using the chat history, simply respond with the answer.
Don't make up content that's not supplied in chat history.
"""


@traceable
def generate_movie_search(chat_history, query):
    messages = (
        [
            {"role": "system", "content": system_prompt},
        ]
        + chat_history
        + [{"role": "user", "content": query}]
    )
    result = openai.chat.completions.create(
        messages=messages, model="gpt-3.5-turbo-0613", tools=tools
    )
    return result.choices[0].message


def _convert_docs(results):
    return [
        {
            "page_content": r,
            "type": "Document",
        }
        for r in results
    ]


@traceable(run_type="retriever")
def retrieve_movies(query):
    # Foo retriever. In production, this would search an actual database
    if "family-friendly" in query.lower():
        return _convert_docs(["Lion King", "Finding Nemo", "The Incredibles"])
    elif "sci-fi" in query.lower():
        return _convert_docs(["Blade Runner 2049", "Interstellar", "The Martian"])
    elif "nature" in query.lower():
        return _convert_docs(["Planet Earth II", "Blue Planet II", "Our Planet"])
    elif "christopher nolan" in query.lower():
        return _convert_docs(["Inception", "Dunkirk", "Interstellar"])
    else:
        return _convert_docs(
            ["Crazy Rich Asians", "The Big Sick", "When Harry Met Sally"]
        )


@traceable
def execute_function_call(message):
    if message.tool_calls[0].function.name == "retrieve_movies":
        query = json.loads(message.tool_calls[0].function.arguments)["query"]
        results = retrieve_movies(query)
    else:
        results = (
            f"Error: function {message.tool_calls[0].function.name} does not exist"
        )
    return results


@traceable
def generate_answer(question, context):
    messages = [
        {
            "role": "system",
            "content": f"Answer the user's question based only on the content below:\n\n{context}",
        },
        {"role": "user", "content": question},
    ]
    result = openai.chat.completions.create(
        messages=messages, model="gpt-3.5-turbo", temperature=0
    )
    return result.choices[0].message.content


@traceable
def rag_pipeline(chat_history, question):
    message = generate_movie_search(chat_history, question)
    if message.tool_calls is None:
        return message.content
    else:
        docs = execute_function_call(message)
        context = "\n".join([doc["page_content"] for doc in docs])
        return generate_answer(question, context)

In [4]:
rag_pipeline([], "what are some family movies to watch?")

'While "Crazy Rich Asians," "The Big Sick," and "When Harry Met Sally" are all great movies, they are not necessarily family-friendly. For family movie night, you may want to consider films like "Finding Nemo," "Toy Story," or "The Incredibles" which are suitable for all ages and enjoyable for the whole family.'